In [ ]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import requests
http = httplib2.Http()
depth_const = 2
from anytree import Node, RenderTree, PreOrderIter
visited_links=set()

def get_contents(link, node):
    #category = 'Anger'
    #link = 'https://vaniquotes.org/wiki/Angry_means'
    page = requests.get(link)
    cid =0
    soup = BeautifulSoup(page.content, "html.parser")
    content_type = soup.find_all("div", {"class": "section"})
    if not content_type: content_type = soup.find_all("div", {"id": "section"})
    all_contents = soup.find_all("div", {"class": "quote"})

    
    for i,content in enumerate(all_contents):
        if len(content_type)==1:
            ref=0
        else: ref=i
        c_type=content_type[ref].find_all(class_="mw-headline")[0].get('id')
        reference = content.get("id") if content.get("id") else content.find_all(class_="quote_link")[0].text

        temp_str=''
        for para in content.find_all("p"):
            temp_str = temp_str + para.get_text()+'\n'
        cid=cid+1
        #list_of_contents.append([c_type, reference, temp_str])
        locals()["child"+str(i)] = Node([c_type, reference, temp_str], parent=node)
    


def recursiveUrl(url, node, depth):
    start=False
    no_of_links=1
    status, response = http.request(url)
    link_list=[]
    for index, link in enumerate(BeautifulSoup(response, parse_only=SoupStrainer('a'))):
        try:
            if link.has_attr('href'):
                #print(link['href'])
                if  "https://vaniquotes.org/w/index.php?" in link['href'] and "from" not in link['href']:
                    start =False

                if start and link['href'].startswith("/wiki/") and no_of_links<=2 and link['href'] not in visited_links:
                    print(url+" Depth - "+ str(depth)+" Link: "+link['href']+" "+str(no_of_links))
                    #print(link['href'])
                    no_of_links=no_of_links+1
                    visited_links.add(link['href'])
                    if '/Category:' not in link['href']:
                        locals()["child_category"+str(index)] = Node(link['href'], parent=node)
                        get_contents("https://vaniquotes.org"+link['href'],locals()["child_category"+str(index)])
                        #link_list.append([url,link['href']])
                        #print(link_list)
                    else:
                        if depth>depth_const:
                            locals()["child_category"+str(index)] = Node(link['href'], parent=node)
                            #link_list.append([[url]+link['href']])
                        else:
                            locals()["child_category"+str(index)] = Node(link['href'], parent=node)
                            list_urls = recursiveUrl("https://vaniquotes.org"+link['href'], locals()["child_category"+str(index)],depth + 1)
                            #link_list.append([[url]+ item for item in list_urls])
                            #link_list.append([list(t) for t in zip([url]*len(list_urls),list_urls)])
                            #print(link_list)

                            #link_list.append(recursiveUrl("https://vaniquotes.org"+link['href'], depth + 1))
                if "https://vanimedia.org/wiki/Main_Page" in link['href']:
                     start =True            

        except:
            pass
    return link_list    

start=False
categories = ["God","krsna","Love","Leadership","Life","Money","Yoga","Time"]#,"Living_Entity_(Jiva)","Duties,_Principles,_Rules,_and_Responsibilities"]


for category in categories:
    url = 'https://vaniquotes.org/wiki/Category:'+category+'_-_Umbrella_Category'
    #url = "https://vaniquotes.org/wiki/Category:God_-_Umbrella_Category"
    udo = Node(url)
    status, response = http.request(url)
    link_list=[]
    no_of_links=1
    category_node = Node(category)
    for index, link in enumerate(BeautifulSoup(response, parse_only=SoupStrainer('a'))):
        try:
            if link.has_attr('href'):
                #print(link['href'])
                if  "https://vaniquotes.org/w/index.php?" in link['href'] and "from" not in link['href']:
                    start =False

                if start and link['href'].startswith("/wiki/") and link['href'] not in visited_links:
                    #print(link['href'])
                    no_of_links=no_of_links+1
                    visited_links.add(link['href'])
                    locals()["child_category_node"+str(index)] = Node(link['href'], parent=category_node)
                    list_urls = recursiveUrl("https://vaniquotes.org"+link['href'],locals()["child_category_node"+str(index)], 0)
                    #link_list.append([[url]+ item for item in list_urls])
                    #link_list.append([list(t) for t in zip([url]*len(list_urls),list_urls)])


                if "https://vanimedia.org/wiki/Main_Page" in link['href']:
                    start =True            

        except:
            print("Error")
            content = [list(leaf.path) for leaf in PreOrderIter(category_node, filter_=lambda node: node.is_leaf)]
            flat_list=[content[i][-1] for i in range(len(content))]
            df = pd.DataFrame(flat_list,columns=["Category"])
            df.to_excel(category+'.xlsx', header=False, index=False)
        
    #flat_list = [x for xs in link_list for x in xs]
    #df = pd.DataFrame(flat_list,columns=["Category","Sub-category","link1"])#,"link2","link3","link4"])
    #df.to_excel(category+'.xlsx', header=False, index=False)
    content = [list(leaf.path) for leaf in PreOrderIter(category_node, filter_=lambda node: node.is_leaf)]
    flat_list=[content[i][-1] for i in range(len(content))]
    df = pd.DataFrame(flat_list,columns=["Category"])
    df.to_excel(category+'.xlsx', header=False, index=False)

https://vaniquotes.org/wiki/Category:Devotees_of_God_-_Umbrella_Category Depth - 0 Link: /wiki/Category:Devotee 1
https://vaniquotes.org/wiki/Category:Devotee Depth - 1 Link: /wiki/Category:Devotee_Incarnation_of_God 1
https://vaniquotes.org/wiki/Category:Devotee_Incarnation_of_God Depth - 2 Link: /wiki/All_the_plenary_manifestations_of_Sankarsana_are_indirect_expansions_of_Lord_Krsna._That_consideration_also_makes_Advaita_Prabhu_an_eternal_servitor_of_Gaura_Krsna._Therefore_He_is_accepted_as_a_devotee_incarnation 1
https://vaniquotes.org/wiki/Category:Devotee_Incarnation_of_God Depth - 2 Link: /wiki/Sri_Caitanya_Mahaprabhu_is_called_bhakta-rupa_(the_form_of_a_devotee),_and_Sri_Nityananda_Prabhu_is_called_bhakta-svarupa_(the_expansion_of_a_devotee)._Sri_Advaita_Prabhu,_the_incarnation_of_a_devotee,_is_visnu-tattva_and_belongs_to_the_same_category 2
https://vaniquotes.org/wiki/Category:Devotee Depth - 1 Link: /wiki/Category:Devotees_of_God 2
https://vaniquotes.org/wiki/Category:Devotees

https://vaniquotes.org/wiki/Category:God%27s_Playing Depth - 2 Link: /wiki/Category:God_Is_Playing_So_Much_Magic 1
https://vaniquotes.org/wiki/Category:God_Is_Playing_So_Much_Magic Depth - 3 Link: /wiki/Yogi_means_that._Yogi_does_not_mean_to_play_some_magic._Magician_also_can_play_some_magic._A_devotee_is_not_interested_to_show_any_magic,_but_he_is_interested_to_see_the_magician,_supreme,_who_is_playing_so_much_magic 1
https://vaniquotes.org/wiki/Category:God%27s_Playing Depth - 2 Link: /wiki/Category:God%27s_Plaything 2
https://vaniquotes.org/wiki/Category:God%27s_Plaything Depth - 3 Link: /wiki/This_material_creation,_consisting_of_the_moving_and_nonmoving,_is_exactly_like_His_plaything._Being_the_Supreme_Lord,_He_is_completely_competent_to_destroy_and_protect 1
https://vaniquotes.org/wiki/Category:Krsna%27s_Playing Depth - 1 Link: /wiki/Category:If_Devotees_of_God_Want_to_Worship_Krsna_in_Conjugal_Love,_Krsna_Perfectly_Plays_the_Part_of_a_Husband_or_Paramour 2
https://vaniquotes.org

https://vaniquotes.org/wiki/Category:Accepting_a_Cheater_as_God Depth - 1 Link: /wiki/Some_people_accept_imitators_and_adversaries_of_the_Supreme_Personality_of_Godhead,_such_as_Paundraka,_Srgala_Vasudeva,_the_spiritual_master_of_the_demons_(Sukracarya),_or_atheists_like_Carvaka,_King_Vena,_Sugata_and_Arhat 2
https://vaniquotes.org/wiki/Category:As_God_-_Umbrella_Category Depth - 0 Link: /wiki/Category:Accepting_Spiritual_Master_As_God 2
https://vaniquotes.org/wiki/Category:Accepting_Spiritual_Master_As_God Depth - 1 Link: /wiki/Acaryam_mam_vijaniyan:_the_Supreme_Personality_of_Godhead_says_that_one_should_respect_the_spiritual_master,_accepting_him_as_the_Lord_Himself._Navamanyeta_karhicit:_one_should_not_disrespect_the_acarya_at_any_time 1
https://vaniquotes.org/wiki/Category:Accepting_Spiritual_Master_As_God Depth - 1 Link: /wiki/As_stated_by_Visvanatha_Cakravarti_Thakura_in_Gurv-astaka,_saksad-dharitvena:_One_directly_accepts_the_guru,_the_spiritual_master,_as_the_Supreme_Personali

https://vaniquotes.org/wiki/Category:God_Is_More_Pleased_When_Devotees_Render_Service_Unto_Him_With_Natural_Affection Depth - 3 Link: /wiki/Although_generally_people_worship_God_with_reverence,_the_Lord_is_more_pleased_when_a_devotee_thinks_of_Him_as_his_pet_son,_personal_friend_or_most_dear_fiance_and_renders_service_unto_Him_with_such_natural_affection 1
https://vaniquotes.org/wiki/Category:Emotions_for_God_-_Umbrella_Category Depth - 0 Link: /wiki/Category:A_Learned_Labor_of_Love_for_the_Sake_of_God 2
https://vaniquotes.org/wiki/Category:A_Learned_Labor_of_Love_for_the_Sake_of_God Depth - 1 Link: /wiki/One_can_attain_the_stage_of_love_of_God_by_pure_devotional_service,_or_by_%22a_learned_labor_of_love,%22_for_the_sake_of_the_devotee%27s_lovable_object,_the_Personality_of_Godhead 1
https://vaniquotes.org/wiki/Category:For_God_-_Umbrella_Category Depth - 0 Link: /wiki/Category:A_Liking_for_God 1
https://vaniquotes.org/wiki/Category:A_Liking_for_God Depth - 1 Link: /wiki/Once_Hanuman,_

https://vaniquotes.org/wiki/Category:God_And..._-_Umbrella_Category Depth - 0 Link: /wiki/Category:Difference_Between_God_And 2
https://vaniquotes.org/wiki/Category:Difference_Between_God_And Depth - 1 Link: /wiki/Category:The_Difference_Between_God_and_Dog 1
https://vaniquotes.org/wiki/Category:The_Difference_Between_God_and_Dog Depth - 2 Link: /wiki/Try_to_understand_the_difference_between_God_and_dog._A_dog_forgets._A_dog_comes_to_your_place_to_eat_something._You_give_it_a_slap._He_goes_away,_again_comes._He_forgets_that_slapping._You_see%3F_That_is_the_dog%27s_nature._And_God%27s_nature_is_different 1
https://vaniquotes.org/wiki/Category:Difference_Between_God_And Depth - 1 Link: /wiki/Category:The_Difference_Between_God_and_Ourselves 2
https://vaniquotes.org/wiki/Category:The_Difference_Between_God_and_Ourselves Depth - 2 Link: /wiki/Because_there_is_a_stage_of_forgetfulness_of_the_individual_selves,_or_the_living_entities,_there_is_always_a_gulf_of_difference_between_the_Lord_and

https://vaniquotes.org/wiki/Category:God_Has_No_Affinity_for_the_Material_Modes Depth - 2 Link: /wiki/Sukadeva_Gosvami_was_already_situated_in_the_liberated_stage,_yet_he_was_attracted_by_the_pastimes_of_the_Lord._This_proves_that_the_quality_of_His_pastimes_has_nothing_to_do_with_material_affinity 2
https://vaniquotes.org/wiki/Category:God_Has..._-_Umbrella_Category Depth - 0 Link: /wiki/Category:Devotees_Completely_Believe_that_God_Has_Inconceivable_Proprietorship_of_Various_Transcendental_Potencies 1
https://vaniquotes.org/wiki/Category:Devotees_Completely_Believe_that_God_Has_Inconceivable_Proprietorship_of_Various_Transcendental_Potencies Depth - 1 Link: /wiki/Devotees_completely_believe,_with_strong_faith,_that_Narayana_is_transcendental_and_has_inconceivable_proprietorship_of_various_transcendental_potencies 1
https://vaniquotes.org/wiki/Category:God_Has..._-_Umbrella_Category Depth - 0 Link: /wiki/Category:God_Has 2
https://vaniquotes.org/wiki/Category:God_Has Depth - 1 Link: /

https://vaniquotes.org/wiki/Category:God_Is_Neither_Pious_Nor_Impious Depth - 2 Link: /wiki/When_a_conditioned_soul_gives_relief_to_others_he_acts_piously,_and_when_he_gives_trouble_to_others_he_acts_impiously,_but_the_Lord_is_neither_pious_nor_impious 2
https://vaniquotes.org/wiki/Category:God_Is_Neither Depth - 1 Link: /wiki/Category:God_Neither_Hates_Nor_Likes_Anyone 2
https://vaniquotes.org/wiki/Category:God_Neither_Hates_Nor_Likes_Anyone Depth - 2 Link: /wiki/The_Lord_neither_hates_nor_likes_anyone,_though_He_appears_to 1
https://vaniquotes.org/wiki/Category:God_Neither_Hates_Nor_Likes_Anyone Depth - 2 Link: /wiki/The_Vedanta-sutras_also_confirm_this:_%22The_Lord_neither_hates_nor_likes_anyone,_though_He_appears_to.%22_BG_1972_purports 2
https://vaniquotes.org/wiki/Category:God_Is_Supreme_-_Umbrella_Category Depth - 0 Link: /wiki/Category:Devotees_of_God_Are_Under_the_Protection_of_God%27s_Supreme_Potency 1
https://vaniquotes.org/wiki/Category:Devotees_of_God_Are_Under_the_Protect

https://vaniquotes.org/wiki/Category:God%27s_-_Umbrella_Category Depth - 0 Link: /wiki/Category:God%27s_Energies_-_Umbrella_Category 1
https://vaniquotes.org/wiki/Category:God%27s_Energies_-_Umbrella_Category Depth - 1 Link: /wiki/Category:God_Does_Not_Know_the_Full_Extent_of_His_Energies 1
https://vaniquotes.org/wiki/Category:God_Does_Not_Know_the_Full_Extent_of_His_Energies Depth - 2 Link: /wiki/God_is_undoubtedly_omniscient,_but_the_personified_Vedas_say_that_even_God_Himself_does_not_know_the_full_extent_of_His_energies._This_does_not_mean_that_God_is_not_omniscient 1
https://vaniquotes.org/wiki/Category:God%27s_Energies_-_Umbrella_Category Depth - 1 Link: /wiki/Category:God_Is_The_Controller_of_All_Energies 2
https://vaniquotes.org/wiki/Category:God_Is_The_Controller_of_All_Energies Depth - 2 Link: /wiki/As_the_controller_of_all_energies,_He_(God)_cannot_at_any_time_be_under_their_influence,_as_we_are 1
https://vaniquotes.org/wiki/Category:God_Is_The_Controller_of_All_Energies Dep

In [34]:
for pre, fill, node in RenderTree(category_node):
    print("%s%s" % (pre, node.name)) 

God
├── /wiki/Category:Devotees_of_God_-_Umbrella_Category
│   ├── /wiki/Category:Devotee
│   │   ├── /wiki/Category:Devotee_Incarnation_of_God
│   │   │   ├── /wiki/All_the_plenary_manifestations_of_Sankarsana_are_indirect_expansions_of_Lord_Krsna._That_consideration_also_makes_Advaita_Prabhu_an_eternal_servitor_of_Gaura_Krsna._Therefore_He_is_accepted_as_a_devotee_incarnation
│   │   │   └── /wiki/Sri_Caitanya_Mahaprabhu_is_called_bhakta-rupa_(the_form_of_a_devotee),_and_Sri_Nityananda_Prabhu_is_called_bhakta-svarupa_(the_expansion_of_a_devotee)._Sri_Advaita_Prabhu,_the_incarnation_of_a_devotee,_is_visnu-tattva_and_belongs_to_the_same_category
│   │   └── /wiki/Category:Devotees_of_God
│   │       ├── /wiki/Category:Devotee_-_different_kinds_of_-_Umbrella_Category
│   │       │   ├── /wiki/Category:Sanskrit_Terms_for_Devotees_of_God_-_Umbrella_Category
│   │       │   │   ├── /wiki/Category:Acarya_-_Devotees_of_God
│   │       │   │   │   ├── /wiki/Category:Follow_the_Acaryas_-_video

In [22]:
from anytree import PreOrderIter
content = [list(leaf.path) for leaf in PreOrderIter(category_node, filter_=lambda node: node.is_leaf)]

In [41]:
len(content)
content[0][-1].values

AttributeError: 'Node' object has no attribute 'values'

In [3]:
from anytree import Node, RenderTree

udo = Node("Udo")
locals()["mar"+str(1)] = Node("Marc", parent=udo)
locals()["mar"+str(2)] = Node("Lian", parent=mar1)
dan = Node("Dan", parent=udo)
jet = Node("Jet", parent=dan)
jan = Node("Jan", parent=dan)
joe = Node("Joe", parent=dan)
krishna =Node("Krishna", parent=mar2)


for pre, fill, node in RenderTree(udo):
    print("%s%s" % (pre, node.name))        

Udo
├── Marc
│   └── Lian
│       └── Krishna
└── Dan
    ├── Jet
    ├── Jan
    └── Joe


In [ ]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import requests
http = httplib2.Http()

threshold = 4
def get_contents(link, category):
    #category = 'Anger'
    link = 'https://vaniquotes.org/wiki/All_the_plenary_manifestations_of_Sankarsana_are_indirect_expansions_of_Lord_Krsna._That_consideration_also_makes_Advaita_Prabhu_an_eternal_servitor_of_Gaura_Krsna._Therefore_He_is_accepted_as_a_devotee_incarnation'
    page = requests.get(link)
    cid =0
    soup = BeautifulSoup(page.content, "html.parser")
    content_type = soup.find_all("div", {"class": "section"})
    if not content_type: content_type = soup.find_all("div", {"id": "section"})
    all_contents = soup.find_all("div", {"class": "quote"})

    list_of_contents=[]
    for i,content in enumerate(all_contents):
        if len(content_type)==1:
            ref=0
        else: ref=i
        c_type=content_type[ref].get("id")
        reference = content.get("id")

        temp_str=''
        for para in content.find_all("p"):
            temp_str = temp_str + para.get_text()+'\n'
        cid=cid+1
        list_of_contents.append([category+'_'+'{num:0{width}}'.format(num=cid, width=6), c_type, reference, temp_str, link])
    return list_of_contents

def content_category(category_link,hierarchy):
    status, response = http.request(category_link[-1])
    start=False
    link_list=[]
    for link in BeautifulSoup(response, parse_only=SoupStrainer('a')):
        try:
            if link.has_attr('href'):
                #print(link['href'])
                if  "https://vaniquotes.org/w/index.php?" in link['href'] and "from" not in link['href']:
                    start =False

                if start and link['href'].startswith("/wiki/"):
                    hierarchy = hierarchy +1
                    if hierarchy < threshold:
                        content_category(category_link+ [link['href']],hierarchy)
                    link_list.append(category_link+ [link['href']])

                if "https://vanimedia.org/wiki/Main_Page" in link['href']:
                    start =True            

        except:
            pass
    return link_list




categories = ["God"]#,"krsna","Love","Leadership","Life","Money","Yoga","Time","Living_Entity_(Jiva)","Duties,_Principles,_Rules,_and_Responsibilities"]


for category in categories:
    category_link = ['https://vaniquotes.org/wiki/Category:'+category+'_-_Umbrella_Category']
    visited_links =[]
    list_of_contents = content_category(category_link,1)
    


    #df = pd.DataFrame(list_of_contents,columns=["ID","Content Type", "Reference","Content","Link"])
    #df.to_excel(category+'.xlsx', header=False, index=False)
              

In [4]:
l1=["a","b","c"]
y=[["d"]]
x=[list(t) for t in zip(["d"]*len(l1),l1)]
#[y[0]+[t] for t in l1]
x

[['d', 'a'], ['d', 'b'], ['d', 'c']]

In [43]:

link = 'https://vaniquotes.org/wiki/All_the_plenary_manifestations_of_Sankarsana_are_indirect_expansions_of_Lord_Krsna._That_consideration_also_makes_Advaita_Prabhu_an_eternal_servitor_of_Gaura_Krsna._Therefore_He_is_accepted_as_a_devotee_incarnation'
page = requests.get(link)
cid =0
soup = BeautifulSoup(page.content, "html.parser")
content_type = soup.find_all("div", {"class": "section"})
if not content_type: content_type = soup.find_all("div", {"id": "section"})
all_contents = soup.find_all("div", {"class": "quote"})

list_of_contents=[]
for i,content in enumerate(all_contents):
    if len(content_type)==1:
        ref=0
    else: ref=i
    c_type=content_type[ref].get("id")
    reference = content.get("id")

    temp_str=''
    for para in content.find_all("p"):
        temp_str = temp_str + para.get_text()+'\n'
    cid=cid+1
    list_of_contents.append([category+'_'+'{num:0{width}}'.format(num=cid, width=6), c_type, reference, temp_str, link])


In [54]:
content_type[0]

<div id="section">
<h2><span class="mw-headline" id="Sri_Caitanya-caritamrta">Sri Caitanya-caritamrta</span></h2>
</div>

In [18]:
def flatten(x1):
    result = []
    for el in x1:
        if hasattr(el, "__iter__") and not isinstance(el, str):
            result.extend(flatten(el))
        else:
            result.append(el)
    return result
z=[]
for item in x:
    z.append(flatten(item))
z

[['d', 'a', 'a1'], ['d', 'b', 'b1'], ['d', 'c', 'c1']]

In [52]:
def content_category(category_link,hierarchy):
    
    for link in BeautifulSoup(response, parse_only=SoupStrainer('a')):
        try:
            if link.has_attr('href'):
                #print(link['href'])
                if  "https://vaniquotes.org/w/index.php?" in link['href'] and "from" not in link['href']:
                    start =False

                if start and link['href'].startswith("/wiki/"):
                    
                    if hierarchy < threshold:
                        content_category(category_link+ [link['href']],hierarchy +1)
                    link_list.append(category_link+ [link['href']])

                if "https://vanimedia.org/wiki/Main_Page" in link['href']:
                    start =True            

        except:
            pass
    return link_list

['https://vaniquotes.org/wiki/Category:God_-_Umbrella_Category',
 '/wiki/Category:God_Categories_to_Change_Names']

In [11]:
category = 'Anger'
link = 'https://vaniquotes.org'+'/wiki/All_the_plenary_manifestations_of_Sankarsana_are_indirect_expansions_of_Lord_Krsna._That_consideration_also_makes_Advaita_Prabhu_an_eternal_servitor_of_Gaura_Krsna._Therefore_He_is_accepted_as_a_devotee_incarnation'

page = requests.get(link)
cid =0
soup = BeautifulSoup(page.content, "html.parser")
content_type = soup.find_all("div", {"class": "section"})
if not content_type: content_type = soup.find_all("div", {"id": "section"})
all_contents = soup.find_all("div", {"class": "quote"})

list_of_contents=[]
for i,content in enumerate(all_contents):
    if len(content_type)==1:
        ref=0
    else: ref=i
    c_type=content_type[ref].get("id")
    reference = content.get("id")

    temp_str=''
    for para in content.find_all("p"):
        temp_str = temp_str + para.get_text()+'\n'
    cid=cid+1
    list_of_contents.append([category+'_'+'{num:0{width}}'.format(num=cid, width=6), c_type, reference, temp_str, link])





In [18]:
mydivs = soup.find_all("div", {"class": "section"})
content = soup.find_all("div", {"class": "quote"})


In [22]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import requests
http = httplib2.Http()


def get_contents(link, category):
    #category = 'Anger'
    #link = 'https://vaniquotes.org/wiki/Angry_means'
    page = requests.get(link)
    cid =0
    soup = BeautifulSoup(page.content, "html.parser")
    content_type = soup.find_all("div", {"class": "section"})
    if not content_type: content_type = soup.find_all("div", {"id": "section"})
    all_contents = soup.find_all("div", {"class": "quote"})

    list_of_contents=[]
    for i,content in enumerate(all_contents):
        if len(content_type)==1:
            ref=0
        else: ref=i
        c_type=content_type[ref].get("id")
        reference = content.get("id")

        temp_str=''
        for para in content.find_all("p"):
            temp_str = temp_str + para.get_text()+'\n'
        cid=cid+1
        list_of_contents.append([category+'_'+'{num:0{width}}'.format(num=cid, width=6), c_type, reference, temp_str, link])
    return list_of_contents

def content_category(category_link, category, visited_links, hierarchy):
    status, response = http.request(category_link)
    start=False
    combined_list_of_contents=[]
    for link in BeautifulSoup(response, parse_only=SoupStrainer('a')):
        try:
            if link.has_attr('href'):
                
                if  "https://vaniquotes.org/w/index.php?" in link['href'] and "from" not in link['href'] :
                    start =False

                if start and link['href'].startswith("/wiki/"):
                    #print(link['href'])
                    if 'Category' in link['href']:
                        if link['href'] not in visited_links:
                            if hierarchy < 3:
                                print(category_link,"Hare Krishna category before",link['href'],hierarchy)
                                list_of_contents = content_category("https://vaniquotes.org"+link['href'], category, visited_links, hierarchy)
                                hierarchy=hierarchy+1
                                print(category_link,"Hare Krishna category",link['href'],list_of_contents,hierarchy)
                                visited_links.append(link['href'])
                                combined_list_of_contents = combined_list_of_contents+list_of_contents
                            else:
                                hierarchy=1
                            #print(category_link,"Hare Krishna category before",link['href'],hierarchy)
                    elif 'wiki' in link['href']:
                        list_of_contents = get_contents("https://vaniquotes.org"+link['href'], category)
                        #print(category_link,"Hare Krishna wiki",link['href'],list_of_contents)
                        #combined_list_of_contents = combined_list_of_contents+list_of_contents
                        #visited_links.append(link['href'])
                    else:
                        print(link['href'])

                if "https://vanimedia.org/wiki/Main_Page" in link['href']:
                    start =True    
        except:
            pass
    return combined_list_of_contents




categories = ["God"]#,"krsna","Love","Leadership","Life","Money","Yoga","Time","Living_Entity_(Jiva)","Duties,_Principles,_Rules,_and_Responsibilities"]


for category in categories:
    category_link = 'https://vaniquotes.org/wiki/Category:'+category+'_-_Umbrella_Category'
    visited_links =[]
    hierarchy=1
    list_of_contents = content_category(category_link, category,visited_links,hierarchy)
    


    df = pd.DataFrame(list_of_contents,columns=["ID","Content Type", "Reference","Content","Link"])
    df.to_excel(category+'.xlsx', header=False, index=False)
              

Svayambhuḥ means Brahma. Brahmā is supposed to be born without any material father and mother. Therefore he is called Svayambhuḥ. He is the only living creature within this universe who is born without the father and mother. How it is? Without the father and mother means this material father and mother. But he has his father. His father is the Supreme Personality of Godhead, Nārāyaṇa. And he's born out of the lotus flower which is grown from the abdomen of Nārāyaṇa. Therefore he's called Svayambhuḥ. Svayambhuḥ means self-manifested. Because before him, there was no existence of material creation. Therefore he is called Svayambhuḥ. And next is Nārada. Svayambhūr nāradaḥ. Nārada is born out of Brahmā. Svayambhūr nāradaḥ and śambhuḥ. Śambhuḥ is also born of Brahmā. Śambhuḥ means Lord Śiva. So he is also one of the ācāryas. Svayambhūr nāradaḥ śambhuḥ kumāraḥ kapilo manuḥ (SB 6.3.20). Kumāraḥ means...
These Kumāras, they are also sons of Brahmā, but they are called Kumāras, brahmacārīs. Whe